# 1 - Téléchargement des librairies et selection du dossier d'images

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install numpy==1.26.1

In [ ]:
images_folder = "images/4"

# 2 - Vérication de cuda (si GPU)

L'utilisation de GPU n'est pas nécessaire pour la création d'embeddings.


In [ ]:
import torch
import numpy as np
from pkg_resources import packaging

torch.cuda.is_available()
print("Torch version:", torch.__version__)

# 3 - Choix du modèle CLIP

In [ ]:
import clip

clip.available_models()

In [ ]:
choix_modele = "ViT-B/32" # ViT-B/32 est le modèle conseillé
nom_modele = choix_modele.replace("/", "_")
print(nom_modele)

# 4 - Création des embeddings
Pour chaque image, l'embedding sera sauvegardé dans un fichier json distinct

In [ ]:
import clip
import torch
from PIL import Image
from io import BytesIO
import json
import os
import time
from tqdm import tqdm


# Charger le modèle CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(choix_modele, device=device)

# Récupérer toutes les images dans le dossier
image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
total = len(image_files)

output_folder = "clip_json"
os.makedirs(output_folder, exist_ok=True)


# Boucle sur chaque image
for idx, filename in enumerate(tqdm(image_files, desc="Traitement des images")):
    input_path = os.path.join(images_folder, filename)
    output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.json")

    # Si le JSON existe déjà, on passe
    if os.path.exists(output_path):
        continue

    try:
        image = Image.open(input_path).convert("RGB")
        image_input = preprocess(image).unsqueeze(0).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image_input)
            image_features /= image_features.norm(dim=-1, keepdim=True)

        data = {
            "filename": filename,
            "embedding": image_features.cpu().tolist()[0]
        }

        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f)


    except Exception as e:
        print(f"[{idx+1}/{total}] ❌ Erreur sur {filename} : {e}")
